In [13]:
%pip install opendatasets --upgrade --quiet

import opendatasets as od
import os
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [4]:
# use either small or large dataset
SMALL_DATASET = True

In [3]:
# insert ypu kaggle  username and key
if (SMALL_DATASET):
    od.download('https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small',data_dir="dataset")
else:
    od.download('https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset',data_dir="dataset")

Skipping, found downloaded files in "dataset\fashion-product-images-small" (use force=True to force download)


In [15]:
if (SMALL_DATASET):
    df_data = pd.read_csv("dataset/fashion-product-images-small/styles.csv", on_bad_lines='skip')
    image_files = os.listdir("dataset/fashion-product-images-small/images")
else:
    df_data = pd.read_csv("dataset/fashion-product-images-dataset/styles.csv", on_bad_lines='skip')
    image_files = os.listdir("dataset/fashion-product-images-dataset/images")

### Clean Data from nulls and non existing images

In [16]:
df_data.isna().sum()

id                      0
gender                  0
masterCategory          0
subCategory             0
articleType             0
baseColour             15
season                 21
year                    1
usage                 317
productDisplayName      7
dtype: int64

In [17]:
df_data.dropna(inplace=True)

In [18]:
#check if file exists
df_data['file_found'] = df_data['id'].apply(lambda x: f"{x}.jpg" in image_files)

In [20]:
df_data['file_found'].value_counts()

file_found
True     44072
False        5
Name: count, dtype: int64

In [22]:
mask = df_data['file_found']
df_data = df_data[mask].reset_index(drop=True)

### Add paths to images and review dataset

In [41]:
# insert ypu kaggle  username and key
if (SMALL_DATASET):
    df_data['filename'] = df_data['id'].apply(lambda x: os.path.join("dataset/fashion-product-images-small/images/",str(x)+".jpg"))
else:
    df_data['filename'] = df_data['id'].apply(lambda x: os.path.join("dataset/fashion-product-images-dataset/images/",str(x)+".jpg"))


In [42]:
df_data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,file_found,filename
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,True,dataset/fashion-product-images-small/images/15...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,True,dataset/fashion-product-images-small/images/39...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,True,dataset/fashion-product-images-small/images/59...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,True,dataset/fashion-product-images-small/images/21...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,True,dataset/fashion-product-images-small/images/53...


In [43]:
#Reduce size of the dataset for test purposes
df_data = df_data.head(2000)

### Convert images to the numpy 3d matrix in order to later use them in the NN.
##### 3d matrix represents width, height and 3 colours

In [44]:
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [45]:
#normalize images by fixin width and height
IMAGE_WIDTH = 50
IMAGE_HEIGHT = 50

In [61]:
images = []
processed_count = 0

In [62]:
def process_image(image_path):
    image = load_img(image_path, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT,3))
    image = img_to_array(image)
    return image

In [64]:
image =load_img(df_data['filename'][0], target_size=(IMAGE_WIDTH, IMAGE_HEIGHT,3))

In [65]:
for file_path in df_data['filename']:
    images.append(process_image(file_path))

In [67]:
images = np.array(images)

In [68]:
images.shape

(2000, 50, 50, 3)

### Split Dataset and Encode Labels

In [72]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [79]:
#for test purposes let's start with 2 lables
output_lables = ["articleType","baseColour"]

In [81]:
(X_train, X_test, 
 y_train,y_test) = train_test_split(images,df_data[output_lables],
                                                    test_size=0.2, random_state=42)

In [82]:
y_train.head()

,articleType,baseColour
968,Kurta Sets,Pink
240,Casual Shoes,Brown
819,Wallets,Red
692,Handbags,Pink
420,Tshirts,Red


In [84]:
lb_article_type = LabelBinarizer()
lb_colour = LabelBinarizer()

article_type_train = lb_article_type.fit_transform(y_train["articleType"].to_numpy())
colour_train = lb_colour.fit_transform(y_train["baseColour"].to_numpy())

article_type_test = lb_article_type.transform(y_train["articleType"].to_numpy())
colour_test = lb_colour.transform(y_train["baseColour"].to_numpy())
